Тема: Линеаризация и дельта-метод

Видео лекции:  
https://www.youtube.com/watch?v=SymKn7KFYKk
    
Видео семинара:  
https://www.youtube.com/watch?v=0QGRAQw4zwM

# Задача к семинару №6: Вычисление линеаризованной метрики

Нужно написать функцию, которая считает пользовательскую линеаризованную метрику.

Допустим, у нас есть информация о покупках пользователей. Мы хотим оценить значимость отличия средней стоимости покупки. Это ratio-метрика, так как мы делим сумму стоимостей покупок на количество покупок.

In [27]:
import pandas as pd
import numpy as np

def calculate_linearized_metric(
    df, value_name, user_id_name, list_user_id, date_name, period, metric_name, kappa=None
):
    """Вычисляет значение линеаризованной метрики для списка пользователей в определённый период.
    
    df - pd.DataFrame, датафрейм с данными
    value_name - str, название столбца со значениями для вычисления целевой метрики
    user_id_name - str, название столбца с идентификаторами пользователей
    list_user_id - List[int], список идентификаторов пользователей, для которых нужно посчитать метрики
    date_name - str, название столбца с датами
    period - dict, словарь с датами начала и конца периода, за который нужно посчитать метрики.
        Пример, {'begin': '2020-01-01', 'end': '2020-01-08'}. Дата начала периода входит в
        полуинтервал, а дата окончания нет, то есть '2020-01-01' <= date < '2020-01-08'.
    metric_name - str, название полученной метрики
    kappa - float, коэффициент в функции линеаризации.
        Если None, то посчитать как ratio метрику по имеющимся данным.

    return - pd.DataFrame, со столбцами [user_id_name, metric_name], кол-во строк должно быть равно
        кол-ву элементов в списке list_user_id.
    """
    # YOUR_CODE_HERE
    
    # Фильтрация пользователей
    df_filtered = df[df[user_id_name].isin(list_user_id)]
    # Фильтрация дат
    df_filtered = df_filtered[(df_filtered[date_name] >= period['begin']) & \
                             (df_filtered[date_name] < period['end'])]
    
    # Расчет числителя и знаменателя
    df_g = df_filtered.groupby(user_id_name).agg(**{
        'x': (value_name, 'sum'),
        'y': (value_name, 'count'),
    }).reset_index()
    
    # Коэффициент линеаризации
    if not kappa:
        kappa = df_g['x'].sum() / df_g['y'].sum()
        
    # Расчет линеаризованной метрики
    df_g[metric_name] = df_g['x'] - kappa * df_g['y']

    # Приводим к нужному числу строк
    df_res = pd.DataFrame({user_id_name: list_user_id}) \
        .merge(df_g[[user_id_name, metric_name]], how='left', on=user_id_name) \
        .fillna(0)
    
    return df_res
    

## Тестирование

In [9]:
dates = pd.date_range(start='2019-12-20', end='2020-01-10')
dates = [str(dt)[:10] for dt in dates]
dates

['2019-12-20',
 '2019-12-21',
 '2019-12-22',
 '2019-12-23',
 '2019-12-24',
 '2019-12-25',
 '2019-12-26',
 '2019-12-27',
 '2019-12-28',
 '2019-12-29',
 '2019-12-30',
 '2019-12-31',
 '2020-01-01',
 '2020-01-02',
 '2020-01-03',
 '2020-01-04',
 '2020-01-05',
 '2020-01-06',
 '2020-01-07',
 '2020-01-08',
 '2020-01-09',
 '2020-01-10']

In [10]:
users = list(range(15))
users

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [18]:
user_col, value_col, date_col = [], [], []
# Генерация датафрейма
for user in users:
    for dt in dates:
        # Число событий для данного пользователя
        n_el = np.random.randint(3)
        for _ in range(n_el):
            user_col.append(user)
            value_col.append(int(np.random.normal(loc=50, scale=10)))
            date_col.append(dt)
            
df = pd.DataFrame({'user_id': user_col, 'value': value_col, 'date': date_col})
df

,user_id,value,date
0,0,46,2019-12-20
1,0,59,2019-12-20
2,0,50,2019-12-21
3,0,64,2019-12-23
4,0,54,2019-12-25
...,...,...,...
329,14,35,2020-01-08
330,14,56,2020-01-09
331,14,50,2020-01-09
332,14,32,2020-01-10


In [8]:
period = {'begin': '2020-01-01', 'end': '2020-01-08'}

In [28]:
calculate_linearized_metric(
    df,
    value_name='value',
    user_id_name='user_id',
    list_user_id=list(range(10)),
    date_name='date',
    period=period,
    metric_name='metric_lin',
    kappa=None)

,user_id,metric_lin
0,0,28.328358
1,1,-3.253731
2,2,35.537313
3,3,30.746269
4,4,-17.253731
5,5,-56.880597
6,6,1.164179
7,7,-18.880597
8,8,16.537313
9,9,-16.044776
